In [6]:
# ! ollama run gemma:7b
# ! ollama run llama3.2

In [1]:
# Imports
import datetime
import time

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import UnstructuredMarkdownLoader

from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


In [3]:
# Load LLM

# MODEL = 'deepseek-r1:14b'
MODEL = 'deepseek-r1:8b'
# MODEL = 'llama3.2'
# MODEL = 'gemma:7b'

llm = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [4]:
def summarize_chapter(file_name, file_name_output, question):

  # Vars
  now = datetime.datetime.now()
  start_time = time.time()
  path_file_load = './book/'+file_name
  path_file_result = './output/'+file_name_output
  path_file_history = './output/history/'+ str(now) + '--' + file_name_output

  # Load file
  loader = UnstructuredMarkdownLoader(path_file_load)
  docs = loader.load()  
  # text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  # splits = text_splitter.split_documents(docs)
  # vectorstore = InMemoryVectorStore.from_documents(
  #     documents=splits, embedding=OpenAIEmbeddings()
  # )

  # Create vector store
  vectorstore = InMemoryVectorStore.from_documents(
    documents=docs, embedding=embeddings
  )

  retriever = vectorstore.as_retriever()

  # Create pronpt
  system_prompt = (
    # "You are an assistant for question-answering tasks. "
    "Use the following chapter to answer the question."
    "If you don't know the answer, say that you don't know. "
    # "Use three sentences maximum and keep the "
    # "answer concise."
    "\n\n"
    "{context}"
  )

  prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    ])

  # Create Pipe
  question_answer_chain = create_stuff_documents_chain(llm, prompt)
  rag_chain = create_retrieval_chain(retriever, question_answer_chain)

  # Execute
  results = rag_chain.invoke({"input": question})

  end_time = time.time()

  # Write
  f = open(path_file_result, "w")
  f.write(results['answer'])
  f.close()

  f = open(path_file_history, "w")
  f.write(f'model: {MODEL}\n')
  f.write(f'start at: {now}\n')
  f.write(f'system_prompt: "{system_prompt.join('; ')}"\n')
  f.write(f'question: {question}\n')
  f.write(f'time to process: {end_time - start_time}\n')
  f.write("\n\n")
  f.write(results['answer'])
  f.close()

In [5]:
summarize_chapter("03-a_motivacao_do_pbb.md", "03-a_motivacao_do_pbb.md", "escreva um resumo detalhado do captulo 'A motivação do PBB' em portugues")

In [7]:
summarize_chapter("04-uma_breve_visao_do_scrum.md", "04-uma_breve_visao_do_scrum.md", "escreva um resumo detalhado do captulo 'Uma breve visão do scrum' em portugues")

In [6]:
summarize_chapter("05-o_protagonista_o_backlog.md", "05-o_protagonista_o_backlog.md", "escreva um resumo detalhado do captulo 'O Protagonista: O Backlog' em portugues")

In [8]:
summarize_chapter("06-product_backlog_building.md", "06-product_backlog_building.md", "escreva um resumo detalhado do captulo 'Product backlog Building' em portugues")

In [9]:
summarize_chapter("07-o_pbb_canvas_e_o_fluxo_de_construcao_do_backlog.md", "07-o_pbb_canvas_e_o_fluxo_de_construcao_do_backlog.md", "escreva um resumo detalhado do captulo 'O PBB canvas e o fluxo de construção do backlog' em portugues")

In [10]:
summarize_chapter("08-coorg_a_tecnica_de_priorizacao_do_pbb.md", "08-coorg_a_tecnica_de_priorizacao_do_pbb.md", "escreva um resumo detalhado do captulo 'COORG: a técnica de priorização do PBB' em portugues")

In [11]:
summarize_chapter("09-pbb_e_backlog_ready.md", "09-pbb_e_backlog_ready.md", "escreva um resumo detalhado do captulo 'PBB e backlog ready' em portugues")

In [12]:
summarize_chapter("10-pbb_e_historia_de_usuario.md", "10-pbb_e_historia_de_usuario.md", "escreva um resumo detalhado do captulo 'PBB e história de usuário' em portugues")

In [13]:
summarize_chapter("11-escreva_historias_de_usuario_com_o_pbb.md", "11-escreva_historias_de_usuario_com_o_pbb.md", "escreva um resumo detalhado do captulo 'Escreva histórias de usuário com o PBB' em portugues")

In [14]:
summarize_chapter("12-pbb_e_taras.md", "12-pbb_e_taras.md", "escreva um resumo detalhado do captulo 'PBB e tapas' em portugues")

In [15]:
summarize_chapter("13-pbb_e_a_lean_inception.md", "13-pbb_e_a_lean_inception.md", "escreva um resumo detalhado do captulo 'PBB e a lean inception' em portugues")